In [4]:
from pathlib import Path
import sh
import time
import datetime

# test

# try:
#     sh.runpkr00()
# except sh.ErrorReturnCode as e:
#     print(f'Command {e.full_cmd} exited with {e.exit_code}')stdout

# try:
#     sh.teqc('+help')
# except sh.ErrorReturnCode as e:
#     print(f'Command {e.full_cmd} exited with {e.exit_code}')stdout

# try:
#     sh.gfzrnx('-h')
# except sh.ErrorReturnCode as e:
#     print(f'Command {e.full_cmd} exited with {e.exit_code}')stdout


In [16]:
cmd = 'gfzrnx'
def is_Commands_sh(Command = cmd):
    try:
        if( Command == 'runpkr00'):
            sh.runpkr00()
        if( Command == 'teqc'):
            sh.teqc('+help')
        if( Command == 'gfzrnx'):
            sh.gfzrnx('-help')
    except sh.ErrorReturnCode as e: 
        print(f'Command {e.full_cmd} exited with {e.exit_code}')

# is_Commands_sh( cmd )    # test    

In [ ]:
# timer:
def startTiming( process_description = 'process' ):
    
    global startTIME
    
    startTIME = time.perf_counter()
    timing = time.strftime("%Y-%m-%d  %H:%M:%S", time.localtime())
    
    print(f'{process_description} starts at: {timing} ')
    
def endTiming( process_description = 'process' ):
    
    global endTIME
    
    endTIME = time.perf_counter()
    timing = time.strftime("%Y-%m-%d  %H:%M:%S", time.localtime())
    costTIME = globals()['endTIME'] - globals()['startTIME']
    
    print(f'{process_description}   ends at: {timing} \n'
          f'{process_description}   costs  : {costTIME/60.0 :7f} min in total ')


In [ ]:
def mkdir_for_srcdir(source_dir, file_dir):
    dirs = {}
    for fdir in file_dir:
        data_dir = Path(f'{source_dir}/{fdir}')
        data_dir.mkdir(mode = 0o777, 
                        parents = True, 
                        exist_ok = True)
        dirs[fdir] = data_dir
    return dirs

# data_dir = mkdir_for_srcdir(src_dir, ['TGD','RNX','TEC'])

def new_obs_info_txt(source_dir = Path( '/home/echoo/data/GNSS/272' ) ,
                     file_list = ['qcinfo', 'RNX_update', 'RNX_rename'], 
                     file_type = 'txt'):
    info_file = {}
    
    for file in file_list:
        file_path = Path(f'{source_dir}/{file}.{file_type}')
        
        if Path.is_file(file_path):
            file_path.touch()
            
        info_file[file] = file_path
    return info_file

# info_path = new_obs_info_txt( src_dir.parent, ['qcinfo',
#                                                'RNX_update',
#                                                'RNX_rename', 
#                                                'RNX_version',] )
# info_path

def delete_file_in_dir( file_to_del_dir ):
    for file_to_del in file_to_del_dir.glob('**/*'):
        if  Path.is_file(file_to_del):
            file_to_del.unlink()


In [ ]:
def cal_obs_ecef_position_teqc(TGD_dir = Path( '/home/echoo/data/GNSS/272/TGD' ),
                               qcinfo_file = info_path['qcinfo'], 
                               update_file = info_path['RNX_update'], 
                               rename_file = info_path['RNX_rename']):
    
    descrip = 'teqc - calculate obs_ecef position'
    
    startTiming( descrip )
    
    with open( update_file, 'w' ) as f_RNXupdate:
        f_RNXupdate.write(f'update_insert:\n'
                          f'#-------------\n'
                          f'  O - ALL:\n'
                          f'    "OBSERVER / AGENCY": {{ '
                          f'0 : "GNSS Observer", '
                          f'1 : "Trimble" }}\n'
                          f'\n'
                          f'  N - ALL:\n'
                          f'    "LEAP SECONDS": {{ 0 : "4" }}\n')
        
    with open( rename_file, 'w' ) as f_RNXrename:
        f_RNXrename.write(f'rename: obs\n'
                          f'#-------------\n'
                          f'  C1 - C2I - C : ALL\n'
                          f'  C6 - C6I - C : ALL\n'
                          f'  C7 - C7I - C : ALL\n'
                          f'  L1 - L2I - C : ALL\n'
                          f'  L6 - L6I - C : ALL\n'
                          f'  L7 - L7I - C : ALL\n')
        
    for count, TGD_file in enumerate( TGD_dir.glob( '*.tgd' ),start = 1 ):
        try:
            sh.teqc( '+qc', '-rep' ,TGD_file ,_out = qcinfo_file )
        except sh.ErrorReturnCode as e:
            print(f'    {descrip} - {TGD_file.stem}: '
                  f'    {descrip} -  exited with {e.exit_code}. ' )
        else:
            print(f'      - {TGD_file.stem}: '
                  f'converted successfully.  ' ) 
        
        with open( qcinfo_file ,'r' ) as f_qcinfo:
            for line in f_qcinfo:
                regex = re.compile( r'4-character ID' )

                if re.search( regex,line ):
                    obs_name = line.split( ':' )[1].split( '(' )[0]

                    with open( update_file, 'a' ) as f_RNXupdate:
                        f_RNXupdate.write( f'\n  O - {obs_name}:\n' )

                regex = re.compile( r'antenna WGS 84 [(]xyz[)]' )

                if re.search( regex,line ):
                    obs_posi = line.split( ':' )[1].split( '(' )[0].split( ' ' )
                    
                    with open( update_file, 'a' ) as f_RNXupdate:
                        f_RNXupdate.write(f'    "APPROX POSITION XYZ": '
                                          f'{{ 0: {obs_posi[1]}, '
                                          f'1: {obs_posi[2]}, '
                                          f'2: {obs_posi[3]} }}\n' )
                    
                    print(f'   {count:4d} - {descrip}: '
                          f'   {obs_name} - WGS84 xyz: {obs_posi[1:4]} .\n' )

                    break
    print( f'   {count} {descrip} in total. /n' )
    endTiming( descrip )


# cal_obs_ecef_position_teqc()
# delete_file_in_dir(data_dir['RNX'])
#     teqc - calculate obs_ecef position   costs  : 26.422665 min in total


In [ ]:
def lla2ecef(lon, lat, alt):
    lla = {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'}
    ecef = {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'}
    transformer = pyproj.Transformer.from_crs(lla,ecef,always_xy = True)
    return transformer.transform(lon,lat,alt,radians = False)

def ecef2lla(x, y, z):
    lla = {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'}
    ecef = {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'}
    transformer = pyproj.Transformer.from_crs(ecef,lla,always_xy = True)
    return transformer.transform(x,y,z,radians = False)